# [EX01]가위바위보 분류기
---

## 이미지 사이즈 조정(28x28)

In [3]:
from PIL import Image
import os, glob


print("PIL 라이브러리 import 완료")

def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
    target_size=(28,28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    print(len(images), " images resized.")

# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/train_image/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/train_image/rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/train_image/paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

#input data resize
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_image/scissor"
resize_images(image_dir_path)
print("테스트 가위 이미지 resize 완료!")

#input data resize
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_image/rock"
resize_images(image_dir_path)
print("테스트 바위 이미지 resize 완료!")

#input data resize
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_image/paper"
resize_images(image_dir_path)
print("테스트 보 이미지 resize 완료!")


PIL 라이브러리 import 완료
2421  images to be resized.
2421  images resized.
가위 이미지 resize 완료!
3037  images to be resized.
3037  images resized.
바위 이미지 resize 완료!
2692  images to be resized.
2692  images resized.
보 이미지 resize 완료!
209  images to be resized.
209  images resized.
테스트 가위 이미지 resize 완료!
209  images to be resized.
209  images resized.
테스트 바위 이미지 resize 완료!
209  images to be resized.
209  images resized.
테스트 보 이미지 resize 완료!


## 각 이미지 및 이미지에 해당하는 라벨 리스트 생성 함수

In [53]:
import numpy as np

def load_data(img_path, number_of_data):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/train_image"
(x_train, y_train)=load_data(image_dir_path, 8150)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_image"
(x_test, y_test)=load_data(image_dir_path, 627)

학습데이터(x_train)의 이미지 개수는 8150 입니다.
학습데이터(x_train)의 이미지 개수는 627 입니다.


## x_train, x_test 이미지의 RGB 각 채널 정규화(/255.0) 

In [54]:
#from sklearn.model_selection import train_test_split
#학습 데이터와 테스트 데이터 나누기
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=1004)

x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))


print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

최소값: 0.0  최대값: 1.0
x_train shape: (8150, 28, 28, 3)
y_train shape: (8150,)


## 모델 설계

In [55]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
n_channel_1=20 # 이미지 특징의 수
n_channel_2=40
n_channel_3=40
n_dense=32 # 분류기에 사용되는 뉴런의 숫자
n_train_epoch=20

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu')) 
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()

Model에 추가된 Layer 개수:  9
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 26, 26, 20)        560       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 13, 13, 20)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 11, 11, 40)        7240      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 5, 5, 40)          0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 3, 3, 40)          14440     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 1, 1, 40)          0         
_________________________________________________________________
flatten_12 (Flatten)         

## 이미지 데이터 reshape

In [57]:
'''
# 해당 프로젝트에 필요 없어 주석 처리
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test.reshape( -1, 28, 28, 3)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))
'''

'\n# 해당 프로젝트에 필요 없어 주석 처리\nprint("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))\nprint("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))\n\nx_train_reshaped=x_train.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.\nx_test_reshaped=x_test.reshape( -1, 28, 28, 3)\n\nprint("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))\nprint("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))\n'

## 학습 및 정확성 테스트

In [56]:
#epochs=10 은 총 데이터를 10번 반복 사용해서 학습하라는 내용
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=n_train_epoch)

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

Epoch 1/20
255/255 [==============================] - 1s 3ms/step - loss: 2.1813 - accuracy: 0.4212
Epoch 2/20
255/255 [==============================] - 1s 3ms/step - loss: 0.7383 - accuracy: 0.6712
Epoch 3/20
255/255 [==============================] - 1s 3ms/step - loss: 0.5469 - accuracy: 0.7730
Epoch 4/20
255/255 [==============================] - 1s 3ms/step - loss: 0.3736 - accuracy: 0.8570
Epoch 5/20
255/255 [==============================] - 1s 3ms/step - loss: 0.3058 - accuracy: 0.8796
Epoch 6/20
255/255 [==============================] - 1s 3ms/step - loss: 0.2482 - accuracy: 0.9043
Epoch 7/20
255/255 [==============================] - 1s 3ms/step - loss: 0.1747 - accuracy: 0.9340
Epoch 8/20
255/255 [==============================] - 1s 3ms/step - loss: 0.1673 - accuracy: 0.9420
Epoch 9/20
255/255 [==============================] - 1s 3ms/step - loss: 0.1336 - accuracy: 0.9491
Epoch 10/20
255/255 [==============================] - 1s 3ms/step - loss: 0.1377 - accuracy: 0.9450

## 회고

> **이번 프로젝트에서 어려웠던 점?**   
> train data 수집이 어려웠고, 정확성을 올리는게 어려웠다.   
> 처음엔 train data 이미지 수량도 부족하였고, train data 안에 잘못된 이미지 들도 많아 학습이 정상적으로 진행되지 못한 것 같다.(~~그냥 배경만 있는 이미지라던가~~)             
> 그러다 보니 정확성이 현저히 낮았다.

> **프로젝트를 진행하면서 알아낸점 혹은 아직 모호한점.**   
> train data 전처리 및 수량의 중요성을 깨달았다.    
> 아직 해당 모델의 구조가 어떤식으로 구동되고, 해당하는 파라미터가 그 구조에서 정확히 어떤 영향을 끼치는지 모호한 것들이 있다.   
> 그래서 하나하나 다 수정해보면서 테스트를 진행하였다.

> **루브릭 평가 지표를 맞추기 위해 시도한 것들.**      
> 1. 하이퍼 파라미터 수정    
> : 이미지 특징의 수(n_channel_1,n_channel_2,n_channel_3) 32 ~ 512 까지 순차적으로 늘리거나 줄이면서 테스트를 진행해봤다.       
> : 분류기에 사용되는 뉴런의 수(n_dense) 16 ~ 128 까지 늘려보았다.       
> : 학습 반복 횟수(n_train_epoch)값을 변경하였다. 내 데이터에서는 20정도 일 때 학습 정확성이 최고값(accuracy: 0.9883)이 나와서 20으로 지정했다.       
> 2. 모델의 층수를 추가했다.    
> 3. train data  와 test data를 추가 했다.   
> : 위의 방법들을 했을 때 정확성이 30프로 정도에서 40프로 대로 올라갔다. 그래서 마지막으로 train data(3000장->8100장), test data(300장->600장)로 수량을 늘리자마자 정확성이 70프로대로 올라갔다.
>
> 초반 train data를 늘리지 않고 이미지의 특징의 수나 분류기에 사용되는 뉴런의 수, 학습 반복 횟수만 올렸을 때는 오버피팅이 되어 오히려 값이 떨어지기도 하였다.

> **자기 다짐**   
> : 아직 해당 모델이 정확히 어떻게 구동되는지 잘 알지 못하여서 해당 모델 공부를 더 할것이다.   
> : 또한 다른 프로젝트를 진행 시 train data와 test data의 품질과 수량을 올리는데 충분한 시간을 투자해야 될 것 같다.     
> : 정확성을 올리기 위해 너무 체계적이지 못하게 테스트를 진행했던 것 같다.(~~노가다로 하이퍼 파라미터 하나하나 바꾸면서~~) 다음 부터는 좀 더 체계적으로 계획을 짜고 테스트를 진행해야 될 것 같다.